In [20]:
import pandas as pd

train = pd.read_csv('./open/train.csv')
test = pd.read_csv('./open/test.csv')
countrywide = pd.read_csv('./open/external_open/countrywide_accident.csv')

data_onehot_enco = pd.read_csv('./open/onehot_encoded_시,일제외.csv', encoding = "UTF_8", low_memory=False)
data_onehot_enco = data_onehot_enco.drop(['Unnamed: 0'], axis=1)
data_onehot_enco.head()

# train_new가 기존 train
# test_new가 기존 test
train_enco = data_onehot_enco[:len(train)]
test_enco = data_onehot_enco[len(train):]
# 동제거
train_drop_dong = [col for col in train_enco.columns if col.startswith("동_")]
train_filtered = train_enco.drop(columns=train_drop_dong, inplace=False)

test_drop_dong = [col for col in test_enco.columns if col.startswith("동_")]
test_filtered = test_enco.drop(columns=test_drop_dong, inplace=False)


# 로그변환 안한 catboost

In [5]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# 여러 개의 피처로 이루어진 데이터
# X_train, y_train은 훈련 데이터
# X_test는 테스트 데이터
# 아래는 예시이므로 실제 데이터에 맞게 수정해야 합니다.

# X_train, y_train 생성
X_train = train_filtered
y_train = train['ECLO']

# train 데이터를 학습(train)과 검증(validation) 세트로 분할
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# CatBoost 모델 생성 및 학습
model = CatBoostRegressor(iterations=1000, depth=10, learning_rate=0.05)
model.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=50, verbose=100)

# # 테스트 데이터 예측
predictions = model.predict(X_valid)
# # 예측 결과를 새로운 데이터프레임에 저장
# result_df = pd.DataFrame({"Predicted_ECLO": predictions})
# # 결과를 파일로 저장
# result_df.to_csv("predicted_eclo.csv", index=False)


def rmsle(y_true, y_pred):
    return np.sqrt(np.mean(np.square(np.log1p(y_true) - np.log1p(y_pred))))

# 테스트 데이터의 실제 값
y_true = y_valid

# 테스트 데이터의 예측 값
y_pred = predictions

# RMSLE 계산
rmsle_value = rmsle(y_true, y_pred)

print("RMSLE:", rmsle_value)

0:	learn: 3.2162359	test: 3.1511960	best: 3.1511960 (0)	total: 5.71ms	remaining: 5.71s
100:	learn: 3.0917664	test: 3.1107327	best: 3.1102988 (94)	total: 537ms	remaining: 4.78s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 3.110298849
bestIteration = 94

Shrink model to first 95 iterations.
RMSLE: 0.4612245376250668


# 로그변환 한 catboost

### 수정예정

In [36]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# X_train, y_train 생성
X_train = train_filtered
y_train = np.log2(train['ECLO'])

# train 데이터를 학습(train)과 검증(validation) 세트로 분할
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# CatBoost 모델 생성 및 학습
model = CatBoostRegressor(iterations=1000, depth=10, learning_rate=0.05)
model.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=50, verbose=100)

# 테스트 데이터 예측
predictions = model.predict(X_valid)

# 테스트 데이터의 실제 값
y_true = y_valid

# 테스트 데이터의 예측 값
y_pred = predictions

def rmsle(y_true, y_pred):
    return np.sqrt(np.mean(np.square(np.log1p(y_true) - np.log1p(y_pred))))

# RMSLE 계산
rmsle_value = rmsle(y_true, y_pred)

print("로그로역변환 전 RMSLE", rmsle_value)

# 예측값 역변환을 위한 로그 역변환
predicted_eclo_back = np.exp2(predictions)

# 추가로, 로그 변환한 값을 다시 원래 스케일로 역변환
y_true_back = np.exp2(y_true)

# RMSLE 계산
rmsle_value_back = rmsle(y_true_back, predicted_eclo_back)

print("로그치환후 RMSLE:", rmsle_value_back)


0:	learn: 0.8346206	test: 0.8367236	best: 0.8367236 (0)	total: 6.59ms	remaining: 6.58s
100:	learn: 0.8050701	test: 0.8272012	best: 0.8271061 (82)	total: 517ms	remaining: 4.6s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.8269429399
bestIteration = 136

Shrink model to first 137 iterations.
로그로역변환 전 RMSLE 0.34036409089089537
로그치환후 RMSLE: 0.4470216363392731


In [33]:
predicted_eclo_back

array([3.46239301, 3.87474725, 4.15462682, ..., 4.03851297, 4.05395248,
       4.38294706])

In [27]:
predictions_sub = model.predict(test_filtered)

In [29]:
predictions_sub = np.exp2(predictions_sub)
predictions_sub

array([3.49977942, 3.31769706, 4.44184515, ..., 4.1716253 , 4.18406941,
       4.03228845])

In [ ]:
# 예측 결과를 새로운 데이터프레임에 저장
result_df = pd.DataFrame({"Predicted_ECLO": predictions_sub})

# 결과를 파일로 저장
result_df.to_csv("predicted_eclo.csv", index=False)